In [ ]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown
from typing import List
from collections import Counter

import os
import sys
sys.path.append('../')
from vibrodiagnostics import mafaulda, selection

MAFAULDA_PATH = '../../datasets/MAFAULDA.zip'
FEATURES_PATH =  '../../datasets/features_data/'
MAFAULDA_METADATA = os.path.join(FEATURES_PATH, selection.MAFAULDA_METADATA)

SHAFT_FAULTS = {'normal': 'N', 'imbalance': 'I', 'horizontal-misalignment': 'HM', 'vertical-misalignment': 'VM'}
BEARING_FAULTS = {'overhang-cage_fault': 'O-Cage', 'underhang-cage_fault': 'U-Cage',
                  'underhang-ball_fault': 'U-Ball', 'overhang-ball_fault': 'O-Ball',
                  'underhang-outer_race': 'U-Race', 'overhang-ball_fault': 'O-Race'}
PLACES = [['ax', 'ay', 'az'], ['bx', 'by', 'bz']]


#FAULT_CLASSES = SHAFT_FAULTS  
FAULT_CLASSES = BEARING_FAULTS

NFFT = 2**14
Fs = mafaulda.FS_HZ

files = pd.read_csv(MAFAULDA_METADATA, index_col='filename')
files.reset_index(inplace=True)

#### Low severity, 2900 RPM

In [ ]:
files = files[files['fault'].isin(tuple(FAULT_CLASSES))]
waveform = files[files['rpm'] >= 2900].groupby(by='fault').first()
waveform

In [ ]:
def time_waveform(files: List[str], time=None, fs=None):
    for filename in files:
        ts = mafaulda.csv_import(ZipFile(MAFAULDA_PATH), filename)
        display(Markdown(f'**{filename}**'))
        
        for axis in PLACES:
            if time is not None:
                start = int(time * fs)
                duration = fs // 10
                dst = ts[axis].iloc[start:start + duration]
            else:
                dst = ts[axis]

            ax = dst.plot(figsize=(20, 8), grid=True, subplots=True)

            for i, axname in enumerate(axis):
                ax[i].set_xlabel('Time [s]')
                ax[i].set_ylabel(f'Amplitude ({axname}) [m/s^2]')
            plt.show() 


def time_waveform_histogram(files: List[str]):
    for filename in files:
        ts = mafaulda.csv_import(ZipFile(MAFAULDA_PATH), filename)
        display(Markdown(f'**{filename}**'))
        
        axis = PLACES[0] + PLACES[1]
        ts[axis].plot.hist(
            figsize=(15, 4), layout=(2, len(axis) // 2), 
            grid=True, subplots=True, 
            bins=100, xlabel='Amplitude'
        ) #, legend=False)
        plt.tight_layout()
        plt.show()


def plot_class_imbalance(labels):
    counter = Counter(labels)
    for k, v in counter.items():
        per = v / len(labels) * 100
        print(f'Class={k}, n={v} ({per:.3f}%%)')
    
    plt.bar(counter.keys(), counter.values())
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()


def spectogram(x):
    fig, ax = plt.subplots(figsize=(15, 4))
    cmap = plt.get_cmap('inferno')
    pxx, freqs, t, im = plt.specgram(
        x, NFFT=NFFT, Fs=Fs,
        detrend='mean',
        mode='magnitude', scale='dB',
        cmap=cmap, vmin=-60
    )
    fig.colorbar(im, aspect=20, pad=0.04)
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Frequency [Hz]')
    mafaulda.resolution_calc(Fs, NFFT)
    return freqs, pxx


def window_idx(t):
    return (Fs * t) // NFFT + 1


def spectrum_slice(freqs, pxx, t):
    fig, ax = plt.subplots(2, 1, figsize=(20, 8))
    n = window_idx(t)

    dB = 20 * np.log10(pxx.T[n] / 0.000001)
    ax[0].plot(freqs, dB)      # 1 dB = 1 um/s^2
    ax[0].grid(True)
    ax[0].set_xlabel('Frequency [Hz]')
    ax[0].set_ylabel('Amplitude [dB]')
    
    ax[1].plot(freqs, pxx.T[n])
    ax[1].grid(True)
    ax[1].set_xlabel('Frequency [Hz]')
    ax[1].set_ylabel('Amplitude [m/s^2]')
    return n


def get_spectrograms(files: List[str], axis: str) -> list:
    spectrograms = []

    for filename in files:
        ts = mafaulda.csv_import(ZipFile(MAFAULDA_PATH), filename)
        display(Markdown(f'**{filename}** ({axis.upper()})'))
        freqs, pxx = spectogram(ts[axis])
        spectrograms.append((filename, freqs, pxx))
        plt.show()

    return spectrograms  


def show_spectrogram_detail(spectrograms: list, axis: str, t: float):
    for name, freqs, Pxx in spectrograms:
        display(Markdown(f'**{name}** ({axis.upper()}) axis @ {t}s)'))
        i_window = spectrum_slice(freqs, Pxx, t)
        plt.show() 

In [ ]:
plot_class_imbalance(files['fault'])

In [ ]:
time_waveform(waveform['filename'])

In [ ]:
time_waveform(waveform['filename'], time=2, fs=mafaulda.FS_HZ)

In [ ]:
time_waveform_histogram(waveform['filename'])

A measurement point

In [ ]:
ax_spectra = get_spectrograms(waveform['filename'], 'ax')

In [ ]:
show_spectrogram_detail(ax_spectra, 'ax', t=2)

In [ ]:
ay_spectra = get_spectrograms(waveform['filename'], 'ay')

In [ ]:
show_spectrogram_detail(ay_spectra, 'ay', t=2)

In [ ]:
az_spectra = get_spectrograms(waveform['filename'], 'az')

In [ ]:
show_spectrogram_detail(az_spectra, 'az', t=2)

B measurement point

In [ ]:
bx_spectra = get_spectrograms(waveform['filename'], 'bx')

In [ ]:
show_spectrogram_detail(bx_spectra, 'bx', t=2)

In [ ]:
by_spectra = get_spectrograms(waveform['filename'], 'by')

In [ ]:
show_spectrogram_detail(by_spectra, 'by', t=2)

In [ ]:
bz_spectra = get_spectrograms(waveform['filename'], 'bz')

In [ ]:
show_spectrogram_detail(bz_spectra, 'bz', t=2)

Features

In [ ]:
# box plots
